# visalizations

> Visualizations to help understand what's going on

In [ ]:
#|default_exp visualizations

In [ ]:
#|hide
%load_ext autoreload
%autoreload 2

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import fastcore.all as fc

from isaacai.utils import *
from isaacai.dataloaders import *
from isaacai.models import *
from isaacai.training import *
import torch.nn as nn
import torch
import matplotlib.pyplot as plt,matplotlib as mpl
from datasets import load_dataset, Dataset
import torchvision.transforms.functional as TF,torch.nn.functional as F

from torcheval.metrics import MulticlassAccuracy
from IPython.display import clear_output

In [ ]:
torch.set_printoptions(precision=2, linewidth=140, sci_mode=False)
torch.manual_seed(1)
mpl.rcParams['image.cmap'] = 'gray'

import logging
logging.disable(logging.WARNING)

set_seed(42)

In [ ]:
xmean,xstd = 0.28, 0.35
@inplace
def transformi(b): b['image'] = [(TF.to_tensor(o)-xmean)/xstd for o in b['image']]

_dataset = load_dataset('fashion_mnist').with_transform(transformi)
_dataset = sample_dataset_dict(_dataset)
dls = DataLoaders.from_dataset_dict(_dataset, 64, num_workers=4)
clear_output()

In [ ]:
trainer = Trainer(dls,
                  nn.CrossEntropyLoss(), 
                  torch.optim.Adam, 
                  SimpleNet(28*28,64,10), 
                  callbacks=[BasicTrainCB(),MetricsCB(Accuracy=MulticlassAccuracy()), DeviceCB()])
trainer.fit()

{'Accuracy': 0.57, 'train_loss': 1.8196, 'valid_loss': 1.3241, 'epoch': 0, 'elapsed': '0:00:01.196352'}
{'Accuracy': 0.66, 'train_loss': 1.1201, 'valid_loss': 1.0177, 'epoch': 1, 'elapsed': '0:00:00.412935'}
{'Accuracy': 0.706, 'train_loss': 0.8555, 'valid_loss': 0.8694, 'epoch': 2, 'elapsed': '0:00:00.413058'}


+ Color Dim
+ Dead Chart
+ Plot Stats

In [ ]:
class ActivationStatsCB:
    
    def color_dim(self): pass
    def dead_chart(self): pass
    def plot_stats(self): pass
    def plot_loss(self:Trainer): pass